In [1]:
import tensorflow as tf
import os
from scipy.io import loadmat
from pyimagesearch import config
import cv2
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import pickle
import matplotlib.pyplot as plt

2024-07-23 22:16:28.259844: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-23 22:16:29.205236: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
image_annotation_map = {}

for category in os.listdir(config.IMAGES_PATH):
    category_image_dir = os.path.join(config.IMAGES_PATH, category)
    category_annotation_dir = os.path.join(config.ANNOTS_PATH, category)
    
    if os.path.isdir(category_image_dir) and os.path.isdir(category_annotation_dir):
        for image_file in os.listdir(category_image_dir):
            image_name, _ = os.path.splitext(image_file) # splits image at '.jpg'
            annotation_name = image_name.replace('image_', 'annotation_')
            annotation_file = annotation_name + '.mat'
            annotation_path = os.path.join(category_annotation_dir, annotation_file)

            if os.path.exists(annotation_path):
                image_path = os.path.join(category_image_dir, image_file)
                image_annotation_map[image_path] = annotation_path


In [4]:
# Example of loading annotation
def load_annotation(annotation_path):
    return loadmat(annotation_path)

In [5]:
example_image_path = next(iter(image_annotation_map))
example_annotation_path = image_annotation_map[example_image_path]
example_annotation = load_annotation(example_annotation_path)
example_annotation

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Thu Dec 23 10:40:59 2004',
 '__version__': '1.0',
 '__globals__': [],
 'box_coord': array([[ 14, 178,   8, 285]], dtype=uint16),
 'obj_contour': array([[ 64.04785894,  80.50377834,  63.697733  ,  69.29974811,
         117.26700252, 180.63979849, 210.05037783, 222.65491184,
         224.40554156, 213.20151134, 233.50881612, 252.76574307,
         255.91687657, 249.26448363, 241.91183879, 262.21914358,
         271.32241814, 272.02267003, 277.27455919, 271.67254408,
         247.5138539 , 213.90176322, 201.29722922, 185.89168766,
         169.43576826, 175.38790932, 168.38539043, 158.23173804,
         123.21914358, 116.91687657, 109.56423174, 116.91687657,
          93.10831234,  82.25440806,  74.55163728,  73.1511335 ,
          64.39798489,  37.7884131 ,  14.68010076,   2.07556675,
          10.47858942,  22.38287154,  38.83879093,  43.39042821,
          36.73803526,  55.29471033,  56.69521411,  64.04785894,
         

In [6]:
example_annotation['box_coord']

array([[ 14, 178,   8, 285]], dtype=uint16)

In [7]:
(x1, y1, x2, y2) = example_annotation['box_coord'][0]
x1, y1, x2, y2

(14, 178, 8, 285)

In [8]:
ex_image = cv2.imread(example_image_path)
ex_image.shape

(226, 300, 3)

In [8]:
# Display the image
cv2.imshow('Image', ex_image)

# Wait for a key press indefinitely or for a specified amount of time in milliseconds
cv2.waitKey(0)

# Close all OpenCV windows
cv2.destroyAllWindows()


In [2]:
# initialize the list of data (images), class labels, target bounding
# box coordinates, and image paths
print("[INFO] loading dataset...")
data = []
labels = []
bboxes = []
imagePaths = []

for category in os.listdir(config.IMAGES_PATH):
    category_image_dir = os.path.join(config.IMAGES_PATH, category)
    category_annotation_dir = os.path.join(config.ANNOTS_PATH, category)
    
    if os.path.isdir(category_image_dir) and os.path.isdir(category_annotation_dir):
        for image_file in os.listdir(category_image_dir):
            image_name, _ = os.path.splitext(image_file) # splits image at '.jpg'
            annotation_name = image_name.replace('image_', 'annotation_')
            annotation_file = annotation_name + '.mat'
            annotation_path = os.path.join(category_annotation_dir, annotation_file)
            
            if os.path.exists(annotation_path):
                image_path = os.path.join(category_image_dir, image_file)
                annotation_data = loadmat(annotation_path)
                (startX, startY, endX, endY) = annotation_data['box_coord'][0]
                image = cv2.imread(image_path)
                (h, w) = image.shape[:2]
                # scale the bounding box coordinates relative to the spatial dimensions of the input image
                startX = float(startX) / w
                startY = float(startY) / h
                endX = float(endX) / w
                endY = float(endY) / h

                # load the image and preprocess it
                image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
                image = tf.keras.preprocessing.image.img_to_array(image)

                # update our list of data, class labels, bounding_boxes and filenames
                data.append(image)
                labels.append(category)
                bboxes.append((startX, startY, endX, endY))
                imagePaths.append(image_path)


[INFO] loading dataset...


In [10]:
data[100].shape, bboxes[100], imagePaths[100], labels[100], len(data)

((224, 224, 3),
 (0.013333333333333334, 0.9267399267399268, 0.02, 1.043956043956044),
 'dataset/images/lobster/image_0005.jpg',
 'lobster',
 6209)

In [11]:
data[0]

array([[[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       ...,

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 2

In [3]:
# convert the data, class labels, bounding boxes, and image paths to
# NumPy arrays, scaling the input pixel intensities from the range [0, 255] to [0, 1]
data = np.array(data, dtype='float32') / 255.0
labels = np.array(labels)
bboxes = np.array(bboxes, dtype='float32')
imagePaths = np.array(imagePaths)

In [4]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# only there are only two labels in the dataset, then we need to use
# Keras/TensorFlow's utility function as well
if len(lb.classes_) == 2:
	labels = tf.keras.utils.to_categorical(labels)

labels.shape

(6209, 97)

In [14]:
labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])

In [5]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing

split = train_test_split(data, labels, bboxes, imagePaths,
                         test_size=0.2, random_state=42)

# unpacj the data split
(X_train, X_test) = split[:2]
(y_train, y_test) = split[2:4]
(trainBBoxes, testBBoxes) = split[4:6]
(trainPaths, testPaths) = split[6:]

# write the testing image paths to disk so that we can use then
# when evaluating/testing our object detector
print("[INFO] saving testing image paths...")
f = open(config.TEST_PATHS, "w")
f.write("\n".join(testPaths))
f.close()

[INFO] saving testing image paths...


In [6]:
# load the VGG16 network, ensuring the head FC layers are left off
vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False,
                                  input_tensor=tf.keras.layers.Input(shape=(224, 224, 3)))

# freeze all VGG layers so they will *not* be updated during the training process
vgg.trainable = False

# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = tf.keras.layers.Flatten()(flatten)

2024-07-23 22:16:45.993558: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-23 22:16:46.183655: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-23 22:16:46.183900: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [7]:
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = tf.keras.layers.Dense(128, activation="relu", kernel_initializer='he_normal')(flatten)
bboxHead = tf.keras.layers.Dense(64, activation="relu", kernel_initializer='he_normal')(bboxHead)
bboxHead = tf.keras.layers.Dense(32, activation="relu", kernel_initializer='he_normal')(bboxHead)
bboxHead = tf.keras.layers.Dense(4, activation="sigmoid",
	name="bounding_box")(bboxHead)
# construct a second fully-connected layer head, this one to predict
# the class label
softmaxHead = tf.keras.layers.Dense(512, activation="relu", kernel_initializer='he_normal')(flatten)
softmaxHead = tf.keras.layers.Dropout(0.5)(softmaxHead)
softmaxHead = tf.keras.layers.Dense(512, activation="relu", kernel_initializer='he_normal')(softmaxHead)
softmaxHead = tf.keras.layers.Dropout(0.5)(softmaxHead)
softmaxHead = tf.keras.layers.Dense(len(lb.classes_), activation="softmax",
	name="class_label")(softmaxHead)
# put together our model which accept an input image and then output
# bounding box coordinates and a class label
model = tf.keras.models.Model(
	inputs=vgg.input,
	outputs=(bboxHead, softmaxHead))

In [8]:
# define a dictionary to set the loss methods -- categorical
# cross-entropy for the class label head and mean absolute error
# for the bounding box head
losses = {
	"class_label": "categorical_crossentropy",
	"bounding_box": "mean_squared_error",
}
# define a dictionary that specifies the weights per loss (both the
# class label and bounding box outputs will receive equal weight)
lossWeights = {
	"class_label": 1.0,
	"bounding_box": 1.0
}
# initialize the optimizer, compile the model, and show the model
# summary
opt = tf.keras.optimizers.Nadam(learning_rate=config.INIT_LR)
model.compile(loss=losses, optimizer=opt, metrics=["accuracy"], loss_weights=lossWeights)
print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 224, 224,  │      1,792 │ input_layer[0][0] │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 224, 224,  │     36,928 │ block1_conv1[0][… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_pool         │ (None, 112, 112,  │          0 │ block1_conv2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv1        │ (None, 112, 112,  │     73,856 │ block1_pool[0][0] │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv2        │ (None, 112, 112,  │    147,584 │ block2_conv1[0][… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 56, 56,    │          0 │ block2_conv2[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv1        │ (None, 56, 56,    │    295,168 │ block2_pool[0][0] │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv2        │ (None, 56, 56,    │    590,080 │ block3_conv1[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv3        │ (None, 56, 56,    │    590,080 │ block3_conv2[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_pool         │ (None, 28, 28,    │          0 │ block3_conv3[0][… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv1        │ (None, 28, 28,    │  1,180,160 │ block3_pool[0][0] │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv2        │ (None, 28, 28,    │  2,359,808 │ block4_conv1[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv3        │ (None, 28, 28,    │  2,359,808 │ block4_conv2[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_pool         │ (None, 14, 14,    │          0 │ block4_conv3[0][… │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5_conv1        │ (None, 14, 14,    │  2,359,808 │ block4_pool[0][0] │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5_conv2        │ (None, 14, 14,    │  2,359,808 │ block5_conv1[0][

 Total params: 31,094,533 (118.62 MB)

 Trainable params: 16,379,845 (62.48 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

None


In [9]:
# construct a dictionary for our target training outputs
trainTargets = {
	"class_label": y_train,
	"bounding_box": trainBBoxes
}
# construct a second dictionary, this one for our target testing
# outputs
testTargets = {
	"class_label": y_test,
	"bounding_box": testBBoxes
}

In [10]:
# train the network for bounding box regression and class label
# prediction
print("[INFO] training model...")
history = model.fit(
	X_train, y_train,
	validation_data=(X_test, y_test),
	batch_size=config.BATCH_SIZE,
	epochs=config.NUM_EPOCHS,
	verbose=1)
# serialize the model to disk
print("[INFO] saving object detector model...")
model.save(config.MODEL_PATH)
# serialize the label binarizer to disk
print("[INFO] saving label binarizer...")
f = open(config.LB_PATH, "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] training model...


: 

In [ ]:
# plot the total loss, label loss, and bounding box loss
lossNames = ["loss", "class_label_loss", "bounding_box_loss"]
N = np.arange(0, config.NUM_EPOCHS)
plt.style.use("ggplot")
(fig, ax) = plt.subplots(3, 1, figsize=(13, 13))
# loop over the loss names
for (i, l) in enumerate(lossNames):
	# plot the loss for both the training and validation data
	title = "Loss for {}".format(l) if l != "loss" else "Total loss"
	ax[i].set_title(title)
	ax[i].set_xlabel("Epoch #")
	ax[i].set_ylabel("Loss")
	ax[i].plot(N, history.history[l], label=l)
	ax[i].plot(N, history.history["val_" + l], label="val_" + l)
	ax[i].legend()
# save the losses figure and create a new figure for the accuracies
plt.tight_layout()
plotPath = os.path.sep.join([config.PLOTS_PATH, "losses.png"])
plt.savefig(plotPath)
plt.close()

In [ ]:
# create a new figure for the accuracies
plt.style.use("ggplot")
plt.figure()
plt.plot(N, history.history["class_label_accuracy"],
	label="class_label_train_acc")
plt.plot(N, history.history["val_class_label_accuracy"],
	label="val_class_label_acc")
plt.title("Class Label Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")
# save the accuracies plot
plotPath = os.path.sep.join([config.PLOTS_PATH, "accs.png"])
plt.savefig(plotPath)